# Hyperparameter search

<img src="https://i.kym-cdn.com/photos/images/newsfeed/000/531/557/a88.jpg">

Or: What if we use machine learning to look for parameters for machine learning models? :-P

(Fun fact: The irony of the situation did occur to many - see the film "Inception"and the model "Inception-v1" - we will talk about it next class.)


## Approaches:
- Manual tuning aka. best practice / “folk wisdom”
    - It can get obsolete in surprisingly short time, eg. [Bengio 2012](https://arxiv.org/abs/1206.5533) suggests values for various hyperparameters based on experience (which is interesting), but still uses prominently "layerwise pretraining"-et, which is totally forgotten by now. (It was quite tedious, it did not bring as much as people hoped for, but noone formally proved it wrong..)
- Scientifically well founded method
    - We simply don't have analytic understanding
    - If we don't have that, how do we know anything if not by simulation? See next point...
- Application of search methods
    - We cast the search for optimal parameters as an optimization problem
    
## Broader context: AutoML

<img src="https://d3ansictanv2wj.cloudfront.net/image1-3250b5283382455a64a9b3fabc39f6c2.jpg" width=55%>

We call **AutoML** all those activities in Data Science whereby we want to automate tasks (not infrequently with machine learning).

Thus can be:
- Automation of data cleaning / feature engineering  eg. [here](https://github.com/featuretools/featuretools/)
- Automatic setting of hyperparameters
- Search for neural components (eg. the Swish activation function we mentioned before : [Searching for Activation Functions](https://arxiv.org/abs/1710.05941))
- Neural architecture search - that is the automatic design of neural "layouts" specifically designed for a given problem. See more [here](https://www.oreilly.com/ideas/what-is-neural-architecture-search)

**For a broader overview it is worth studying the recent paper [AutoML: A Survey of the State-of-the-Art](https://arxiv.org/abs/1908.00709).**

<img src="http://drive.google.com/uc?7export=view&id=1xp2zv7RJCQWA7BAQP7nsoS6A1Ugko5jO" width=85%>

## How does this search space look like?
- Individual evaluation steps are very expensive (even in case of short “trial runs”)
- We do not know much about the space, we don't think it is convex it can have non-trivial interactions between the dimensions, though there ougth to be wider "plateaus" with good solutions
- Exploration / exploitation tradeoff
- Albeit very well parallelizable even at first glance

## Gridsearch

We create a hypercube in the parameter space which we divide into unified regions (still tolerable in number), choose a point in each region (a combination of parameters) and then iteratively start training with that combination.

<img src="https://i.stack.imgur.com/02p4O.png" width=450 heigth=450 align="left">
<img src="https://i.imgur.com/MsYDib8.jpg" width=450 heigth=450 align="right" style="float:left;margin:0 10px 10px 0">


### Simple gridsearch example



In [ ]:
import itertools
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

log_dir = "logs"

model_path = log_dir + "/model.ckpt"

# Task parameters

input_size = 784
n_classes = 10

# Data 

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
n_train = np.shape(mnist.train.images)[0]

def create_and_train_model(n_hidden_layers=3,
                           hidden_layer_size=128,
                           initial_lr=0.5,
                           batch_size=100,
                           constant_lr_epochs = 15,
                           lr_decay = 0.8,
                           early_stopping_patience = 3,
                           drop_out_keep = 0.8,
                           train_writer_path=log_dir + "/train",
                           valid_writer_path=log_dir + "/valid",
                           model_path=model_path,
                           save_below_valid_loss=np.inf):
    "Create and train a model with the given parameters."
    
    tf.reset_default_graph()

    # Input

    keep_prob = tf.placeholder_with_default(1.0, shape=(), name="keep_prob")
    learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
    X = tf.placeholder(tf.float32, shape=[None, input_size], name="X")
    Y = tf.placeholder(tf.float32, shape=[None, n_classes], name="Y")

    # Model internals

    # hidden layers

    hidden_layer_sizes =  n_hidden_layers * [hidden_layer_size]
    layer_sizes = [input_size] + hidden_layer_sizes
    last_layer = X

    for n in range(len(hidden_layer_sizes)):
        cur_layer_input_size = layer_sizes[n]
        cur_layer_output_size = layer_sizes[n+1]
        weights = tf.Variable(tf.random_normal([cur_layer_input_size, cur_layer_output_size]),
                              name="W_hidden_" + str(n))
        bias = tf.Variable(tf.random_normal([cur_layer_output_size]),
                           name="b_hidden_" + str(n))
        activations = tf.sigmoid(last_layer @ weights + bias)
        last_layer = tf.nn.dropout(activations, keep_prob=keep_prob)

    # Output layer 
    
    W = tf.Variable(tf.random_normal([cur_layer_output_size, n_classes]), name="W")
    b = tf.Variable(tf.random_normal([n_classes]), name="b")
    logits = tf.add(last_layer @ W, b, name="logits")

    # Loss and optimizer

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits),
                          name="loss")
    train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

    # Accuracy

    correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(logits,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

    # Init, save & summary opts

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    accuracy_summary = tf.summary.scalar("Accuracy_Summary", accuracy)
    loss_summary = tf.summary.scalar("Loss_Summary", loss)
    merged = tf.summary.merge_all()
    
    # Training
    ##########

    train_writer = tf.summary.FileWriter(train_writer_path, graph=tf.get_default_graph())
    valid_writer = tf.summary.FileWriter(valid_writer_path, graph=tf.get_default_graph())
    n_epoch_steps = n_train // batch_size
    min_valid_loss = np.inf
    valid_accuracy_for_min_loss = 0
    n_deteriorating_losses = 0
    step = 1
    epoch = 1
    lr = initial_lr

    with tf.Session() as sess:
        sess.run(init)
        while True:
            batch = mnist.train.next_batch(batch_size)
            _ = sess.run([train_step],
                         feed_dict={X: batch[0],
                                    Y: batch[1],
                                    keep_prob: drop_out_keep,
                                    learning_rate: lr})
            if step % n_epoch_steps == 0: # we are at the end of an epoch
                train_loss_val, train_accuracy_val, train_summary \
                    = sess.run([loss, accuracy, merged],
                               feed_dict={X: mnist.train.images,
                                          Y: mnist.train.labels})
                train_writer.add_summary(train_summary, step)
                valid_loss_val, valid_accuracy_val, valid_summary \
                    = sess.run([loss, accuracy, merged],
                               feed_dict={X: mnist.validation.images,
                                          Y: mnist.validation.labels})
                valid_writer.add_summary(valid_summary, step)
                print(f"Epoch {epoch}. Train loss: {train_loss_val}, train_accuracy: {train_accuracy_val}, valid loss: {valid_loss_val}, valid accuracy: {valid_accuracy_val}.")
                if valid_loss_val >= min_valid_loss:
                    n_deteriorating_losses += 1
                    print("Deteriorating loss no.", n_deteriorating_losses)
                    if n_deteriorating_losses == early_stopping_patience:
                        print("Patience's run out, stopping.")
                        break
                else:
                    n_deteriorating_losses = 0
                    min_valid_loss = valid_loss_val
                    valid_accuracy_for_min_loss = valid_accuracy_val
                    if min_valid_loss < save_below_valid_loss:
                        print(f"Saving the model since new loss {min_valid_loss} is the global minimum so far.")
                        saver.save(sess, model_path)
                epoch += 1
                if epoch > constant_lr_epochs:
                    lr *= lr_decay
                    print(f"Learning rate set to {lr}.")
            step += 1
        print(f"The best loss on the the validation set was {min_valid_loss}.")
        return min_valid_loss, valid_accuracy_for_min_loss

    
def grid_search(layer_sizes, layer_ns, initial_learning_rates):
    "Perform a grid search with the given parameter lists."
    min_valid_loss = np.inf
    configs = itertools.product(layer_sizes, layer_ns, initial_learning_rates)
    for layer_size, layer_n, initial_lr in configs:
        print("=====================================")
        print(f"Training model with layer size {layer_size}, layer number {layer_n} and initial learning rate {initial_lr}.")
        cur_model_name = f"ls{layer_size}_ln{layer_n}_ilr{initial_lr}"
        cur_valid_loss, \
            cur_valid_acc = create_and_train_model(n_hidden_layers=layer_n,
                                                   hidden_layer_size=layer_size,
                                                   initial_lr=initial_lr,
                                                   train_writer_path=log_dir + "/" + cur_model_name + "_train",
                                                   valid_writer_path=log_dir + "/" + cur_model_name + "_valid",
                                                   model_path=model_path,
                                                   save_below_valid_loss=min_valid_loss)
        if cur_valid_loss < min_valid_loss:
            min_valid_loss = cur_valid_loss
            accuracy_for_min_valid_loss = cur_valid_acc
            best_model_name = cur_model_name
        print(f"The current minimal valid loss is {min_valid_loss} with {accuracy_for_min_valid_loss} accuracy, produced by model {best_model_name}.")


# Perform the actual grid search

grid_search([32, 64, 128], [1, 2, 3], [1., 0.1, 0.01])

# Measure performance on the test data set

tf.reset_default_graph()
saver = tf.train.import_meta_graph(model_path + ".meta")

with tf.Session() as sess:
     print("Restoring the best model to test it.")
     saver.restore(sess, model_path)
     test_loss_val, test_accuracy_val = sess.run(["loss:0", "accuracy:0"],
                                                 feed_dict={"X:0": mnist.test.images,
                                                            "Y:0": mnist.test.labels})
     print(f"The loss on the the test set is {test_loss_val}.")
     print(f"The accuracy on the test set is {test_accuracy_val}.")


### Valid loss on different models

<img src="http://drive.google.com/uc?export=view&id=1-7aZgdcItHFmt9lNlwBCxzN6cpL88pfU">



## Random search 

We would think, that randomly trying combinations is not that fruitful.
In fact, we are not that lucky to be right.
[Bergstra and Bengio 2012](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf) analyzed the question, and came up with the surprising result, that doing random search is a bit more effective in many cases than a grid based solution (given the same computational budget).

<img src="http://drive.google.com/uc?export=view&id=13aQyREuB9eEJBRsR1ExLbquB2sz5-zvJ" width=70%>


## “Exotic” optimalization solutions

**They are not exotic at all, we can just not talk about everything. There are complete paradigms out there that we did not discuss, and can any time make a comeback!**

Our starting point: We ought to do better than random search!

What can we use?

### Bayesian learning

We can utilize the paradigm of bayesian learning for the search of hyperparameters.

We did not go into detail about the bayesian methods of probability, so we don't discuss this further here. Some intorductory resources can be found [here](https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f).


### "Evolutionary computation" / "Genetic Algorithms"

Also a biologically inspired computational method, which is based on the notion of "inheritance" and selection of good solution (parts).

#### Main types:

<img src="http://slideplayer.com/slide/3200475/11/images/7/What+are+the+different+types+of+EAs.jpg" width=500 heigth=500>

We only detail this set of learning methods, because people applied it to hyperparameter search, or more importantly to architecture search under the name of "neuroevolution". SOme researchers created a domain specific language that defines possible structure candidates for neural networks, over which genetic methods can optimize. See more [here](https://arxiv.org/abs/1801.01563).

#### Basics of genetic algorithms

We describe one possible solution to a problem with a "chromosome" of binary values, where each location defines one "choice" in a space of settings.

<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/07/22170200/Capture1-300x186.png">

If we relax the binary assumption and allow for the usage of scalar values also, we can easily cast the hyperparameter search problem as a "chromosome" of categorical and scalar values representing choices and settings for the training.

The only thing we have to pay attention to is to ensure "conformance" during the genetic operations so as all chromosomes can be rightly interpreted as parameter combinations. This is usually pretty easy.

##### Algorithm

<img src="http://www.pohlheim.com/Papers/mpga_gal95/fig1.gif">

##### Fitness

Again - and rather unsurprisingly - we need some kind of objective function for optimization, which is called "fitness" in this domain. This behaves the same as any cost function, but we have no restrictions on it, it does not have to be differentiable or even very smooth. The only criterion: in general it should allow for some kind of ranking.

##### Selection

It could be easily Top N - and many times it is - but if we aim to balance diversity with "pressure" towards goodness and avoiding "monoculture", we can choose more complicated methods, like tournament selection, though diversity is also strongly influenced by crossover and mutation operators.
<img src="http://slideplayer.com/slide/5231384/16/images/35/Example:+Tournament+selection.jpg" width=65%>

<img src="https://i.stack.imgur.com/zBHSp.png" width=65%>

##### Combination / "crossover"

<img src="https://image.slidesharecdn.com/seminaronga-1228280582477612-9/95/genetic-algorithms-made-easy-12-728.jpg?cb=1378336663" width=500 heigth=500>

##### Mutation

<img src="https://lh6.ggpht.com/_NNjxeW9ewEc/TNrUJ0Z8KlI/AAAAAAAARKg/zcS23GZlcvw/tmp1C17_thumb_thumb.jpg?imgmax=800" width=500 heigth=500>

Please observe the tree structure! GA-s can rather elegantly represent decision trees for example!

It is also owrth noting, that in the example above,the structure itself changed, since we switched to an operator with two arguments, so if we handle this right, we can cause structural change also and we may not need crossover at all, thus we simulate "asexual reproduction".

#### Advantages

- Parallel and distributed solutions appeared rather early, thus quite unsophisticated hardware could support the training.
Let me show you! :-)

<img src="http://drive.google.com/uc?export=view&id=0B2Mh1BtG1i-JMTJCQTY0QTFENjAxNUU4QjowLjE" width=50%>

- It can easily and naturally represent categorical values, it can handle scalars, it is very flexible. It can incorporate many constraints which can be part of the representation as well as the operations.

Summary: **Evolutionary computation is not dead, just sleeping!**

#### Genetic algorithm basic example

There is a pretty straightforward example for using GA for hyperparameter search:

[Let's evolve a neural network with genetic algorithm](https://blog.coast.ai/lets-evolve-a-neural-network-with-a-genetic-algorithm-code-included-8809bece164)

It tunes four parameters:
- Number of layers (depth)
- Neurons per layer (width)
- Activation function
- Type of optimizer (SGD or else)

The code can be found [here](https://github.com/harvitronix/neural-network-genetic-algorithm/blob/master/train.py)

#### Evolutionary search as a direct optimizer for neural networks

Interestingly, evolutionary methods can be also used directly for the gradient free optimization of neural models. This solution can be rather interesting in situations where reinforcement learning methods would be the only options. For a long time, evolutionary methods were forgotten, but there is now some kind of revival going on.

If we compare or even better COMBINE gradient based methods with evolutionary ones, we can have better results! (Think: combination of gradients and mutation, different behavior in case of local gradient gaps, robustness...)

In [ ]:
from IPython.display import HTML
HTML("""<video controls=""  loop="" preload="metadata" class="arve-video fitvidsignore"><source type="video/mp4" src="https://1fykyq3mdn5r21tpna3wkdyi-wpengine.netdna-ssl.com/wp-content/uploads/2017/12/gradient_gap_composite.mp4"></video>
""")


But well, it also has some drawbacks...

In [ ]:
HTML("""<video controls="" controlslist="nodownload" loop="" preload="metadata" class="arve-video fitvidsignore"><source type="video/mp4" src="https://1fykyq3mdn5r21tpna3wkdyi-wpengine.netdna-ssl.com/wp-content/uploads/2017/12/narrowing_path_composite.mp4"></video>""")

**Well worth reading further [here](https://eng.uber.com/deep-neuroevolution/).**

### Reinforcement learning

In case of reinforcement learning we have "sparse teaching signal", so not every input has an associated "label".

**This is a HUGE  paradigm in itself, pretty cutting edge**, but sadly out of scope for the curent course.

Google “AutoML” has some Reinforcement Learning solutions in it.
More details [here](https://research.googleblog.com/2017/05/using-machine-learning-to-explore.html) and [here]( https://arxiv.org/abs/1611.02167).


Some recent news about AutoML [here](https://news.mit.edu/2019/convolutional-neural-network-automation-0321#.XJeRl6BoKM4.facebook).

### Other interesting approaches

There are other paradigms inspired by nature, like: ["Particle swarm"](http://www.swarmintelligence.org/tutorials.php) vagy ["Ant colony"](https://en.wikipedia.org/wiki/Ant_colony_optimization_algorithms) methods.

And there is a remarkable **global optimization method**:

[A Global Optimization Algorithm Worth Using](http://blog.dlib.net/2017/12/a-global-optimization-algorithm-worth.html)

In [ ]:
HTML("""<video controls="" controlslist="nodownload" loop="" preload="metadata" class="arve-video fitvidsignore"><source type="video/webm" src="http://dlib.net/find_max_global_example.webm"></video>""")

# Famous last words:

It also could have occured to us in case of previous models - eg. RandomForest and co. - that setting the hyperparameters is non-trivial.
Naturally there was a need for an "out of the box" solution: [Sklearn-Hyperopt](https://github.com/hyperopt/hyperopt-sklearn)

Where there is one solution, there is many more :-)
- [auto-sklearn](https://automl.github.io/auto-sklearn/master/)
- [NeuPy](http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html)
- [Sklearn-DEAP](https://github.com/rsteca/sklearn-deap)
- [Optunity](https://github.com/claesenm/optunity)
- [TPOT](https://github.com/EpistasisLab/tpot)
- [Karoo gp](https://kstaats.github.io/karoo_gp/)
- [Spearmint](https://github.com/JasperSnoek/spearmint)
- [NEAT](https://github.com/CodeReclaimers/neat-python)
- [McFly](https://github.com/NLeSC/mcfly/wiki/Technical-documentation)
- [FAR-HO](https://github.com/lucfra/FAR-HO)

There are long lists of libraries for example [here](https://medium.com/@mikkokotila/a-comprehensive-list-of-hyperparameter-optimization-tuning-solutions-88e067f19d9). This is an infinite subject.

In fact, there is also a combined library for **Neural Architecture Search and hyperparameter optimization** for "the laymen" called [Autokeras](https://github.com/keras-team/autokeras). This area is definitely getting commoditized!